In [1]:
import torch as th
import syft as sy
import torch.nn as nn
hook = sy.TorchHook(th)

W0729 22:56:01.823956 140461298792256 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/zeemon/anaconda3/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0729 22:56:01.857664 140461298792256 deprecation_wrapper.py:119] From /home/zeemon/anaconda3/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
workers1 = dict()
for i in range(20):
    workers1['worker{i}'.format(i=i)] = sy.VirtualWorker(hook, id='worker{i}'.format(i=i))

In [3]:
data = 10*th.rand(40,2, requires_grad=True)
tar = list()
for i in data:
    tar.append(1 if(i.sum() > 10) else 0)
target = th.Tensor(tar)

In [4]:
for ii, i in enumerate(range(0, 40, 2)):
	workers1['worker{ii}'.format(ii=ii)].clear_objects()
	workers1['dataworker{ii}'.format(ii=ii)] = data[i:i+50].send(workers1['worker{ii}'.format(ii=ii)])
	workers1['targetworker{ii}'.format(ii=ii)] = target[i:i+50].send(workers1['worker{ii}'.format(ii=ii)])

In [5]:
data_target_set = list()
for i in range(20):
	data_target_set.append(tuple((workers1['dataworker{ii}'.format(ii=i)], 
						workers1['targetworker{ii}'.format(ii=i)])))

In [7]:
def train(iterations=20, lr=0.001):

    model = nn.Linear(2,1)
    opt = th.optim.SGD(params=model.parameters(), lr=lr)

    for iter in range(iterations):

        for _data, _target in data_target_set:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()
            print(loss.get())

In [8]:
train()

tensor(7265.9565, requires_grad=True)
tensor(1.3155e+08, requires_grad=True)
tensor(2.3792e+12, requires_grad=True)
tensor(3.4510e+16, requires_grad=True)
tensor(3.9316e+20, requires_grad=True)
tensor(3.5654e+24, requires_grad=True)
tensor(2.6843e+28, requires_grad=True)
tensor(1.5806e+32, requires_grad=True)
tensor(6.7792e+35, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(inf, requires_grad=True)
tensor(nan, requires_grad=True)
tensor(nan, requires_grad=True)
tensor(nan, requires_grad=True)
tensor(nan, requires_grad=True)
tensor(nan

In [9]:
train(lr=0.0000001)

tensor(69038.4844, requires_grad=True)
tensor(58608.9375, requires_grad=True)
tensor(50525.9922, requires_grad=True)
tensor(44285.3633, requires_grad=True)
tensor(37959.9805, requires_grad=True)
tensor(33936.8945, requires_grad=True)
tensor(30903.5586, requires_grad=True)
tensor(26147.7812, requires_grad=True)
tensor(21930.9922, requires_grad=True)
tensor(18491.6211, requires_grad=True)
tensor(14179.8955, requires_grad=True)
tensor(10684.6719, requires_grad=True)
tensor(8270.3096, requires_grad=True)
tensor(5808.6875, requires_grad=True)
tensor(4524.3657, requires_grad=True)
tensor(3559.2737, requires_grad=True)
tensor(2162.2136, requires_grad=True)
tensor(1351.4978, requires_grad=True)
tensor(494.1292, requires_grad=True)
tensor(80.8687, requires_grad=True)
tensor(54769.3086, requires_grad=True)
tensor(46420.0703, requires_grad=True)
tensor(40019.9141, requires_grad=True)
tensor(35106.7852, requires_grad=True)
tensor(30083.5625, requires_grad=True)
tensor(26918.1562, requires_grad=Tru